In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error

from src.constants import *

from src.utility.personal import *
from src.utility.structure import *
from src.utility.storage import *
from src.utility.math import *

from src.avgs import *
from src.trajectory import *

from src.rolling_data import RollingData
from src.per_game_model import PerGameModel
from src.init_DFs.per_game import PerGameInit
from src.init_DFs.next_game import NextGameInit

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

pgModel = PerGameModel()
initPG = PerGameInit()
initNextGame = NextGameInit()

In [2]:
season = "all"
situation = "5on5"
df = primary_csv(season, situation)

num_games = 4

In [3]:
df_B = trajectory_linear(df.copy(), num_games, add_objs=["season", "game_number"], suffix=False)

In [6]:
b1=df_B[1]
b0=df_B[0]
b1

,game_id,team,opp_team,season,game_number,next_game_id,score,opp_score,win,reg_win,...,scoreAdjustedShotsAttempts,unblockedShotAttempts,scoreAdjustedUnblockedShotAttempts,dZoneGiveaways,xGoalsFromxReboundsOfShots,xGoalsFromActualReboundsOfShots,reboundxGoals,totalShotCredit,scoreAdjustedTotalShotCredit,scoreFlurryAdjustedTotalShotCredit
0,2014-10-13BOSCOL,BOS,COL,2014,4,2014-10-15DETBOS,-0.4,0.5,-0.3,-0.3,...,-0.1251,1.2,0.5213,0.1,0.0254,0.0373,0.0301,0.2499,0.2184,0.2155
1,2014-10-13TBLMTL,MTL,TBL,2014,4,2014-10-16MTLBOS,-0.7,1.4,-0.3,-0.3,...,0.7922,-0.7,-1.6482,-1.8,0.0075,-0.1731,-0.1959,0.1126,0.0692,0.0711
2,2014-10-14CARBUF,BUF,CAR,2014,4,2014-10-17BUFFLA,0.8,-0.1,0.3,0.0,...,-1.8783,-3.8,-3.1249,-0.9,-0.0668,-0.0354,-0.0354,-0.5264,-0.4768,-0.4710
3,2014-10-14LAKEDM,LAK,EDM,2014,4,2014-10-16LAKSTL,2.0,-1.1,0.4,0.4,...,-1.9799,-2.2,-0.5421,0.4,0.0003,0.0539,0.0539,0.1850,0.2408,0.2424
4,2014-10-14NSHCGY,CGY,NSH,2014,4,2014-10-15CHICGY,-0.1,-0.4,0.2,-0.1,...,4.5612,0.4,0.9468,-1.2,-0.0015,0.0201,0.0087,-0.1123,-0.0858,-0.0858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20707,2023-04-13WSHNJD,WSH,NJD,2022,82,NaN,0.3,0.6,-0.1,-0.1,...,2.0685,3.4,3.8122,-1.4,0.0190,0.0829,0.0829,0.1037,0.1326,0.1390
20708,2023-04-14CBJBUF,BUF,CBJ,2022,82,NaN,0.8,-0.3,0.1,0.3,...,6.7506,3.7,4.2156,-0.2,0.0574,0.0355,0.0355,0.3667,0.3850,0.3757
20709,2023-04-14CBJBUF,CBJ,BUF,2022,82,NaN,0.6,0.1,0.1,0.0,...,4.5546,5.1,5.2618,-0.2,0.0720,0.4211,0.4211,0.3727,0.3693,0.3696
20710,2023-04-14NSHCOL,COL,NSH,2022,82,NaN,0.0,-0.3,0.1,0.4,...,1.5919,-0.1,0.3627,1.2,0.0113,0.0283,0.0283,0.1221,0.1540,0.1439


In [7]:
export_csv(b0.copy(), season, situation, 
           name=f'LINEAR_b0_{num_games}', subfol="TRAJECTORY", 
           pretty=True)
export_csv(b1.copy(), season, situation, 
           name=f'LINEAR_b1_{num_games}', subfol="TRAJECTORY", 
           pretty=True)